Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 6.95 ms, sys: 344 µs, total: 7.29 ms
Wall time: 11.5 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.1.0.4/6938/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 6 Memory: 7.29 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 261 ms, sys: 34.8 ms, total: 296 ms
Wall time: 2.23 s


[1.372607780797988,
 1.030681105966792,
 1.6540620215695712,
 1.2175141599316892,
 1.9942287817171152,
 1.6836515444987232,
 1.9791573938184435,
 1.0050377827807642,
 1.0617325696060782,
 1.6068733235916421,
 1.4964064363694964,
 1.2804550571983753,
 1.3485774712477179,
 1.533843793948546,
 1.1294485122820435,
 1.5359639908709881,
 1.742854668375849,
 1.6319723325386306,
 1.9384224065674063,
 1.4299092043033008,
 1.125053456777521,
 1.9410548915213082,
 1.6277306496164987,
 1.5992637289214144,
 1.2817784432339843,
 1.6494006606957117,
 1.4219802586583878,
 1.6250279738342246,
 1.6352358869945776,
 1.5191281974379218,
 1.4033624173522345,
 1.3854736314827494,
 1.1473575766209794,
 1.7684906462363865,
 1.4852997040144014,
 1.7881834440931255,
 1.555495861494375,
 1.1261319567306236,
 1.1393461690319606,
 1.3761336925778633,
 1.6217377703076759,
 1.729938614731991,
 1.0838848451879213,
 1.737956550022896,
 1.4502705893400964,
 1.2995124651458916]

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.6475583669778846,
 1.769770970617118,
 1.1340021246808991,
 1.1864395210828929,
 1.6444528709731046,
 1.528857600018179,
 1.2576590217209853,
 1.2817515535530972,
 1.189286186314415,
 1.2711641710547603,
 1.4546061554732441,
 1.6895611039602687,
 1.4595488941171912,
 1.9006835446860642,
 1.3453222541239849,
 1.270363297143236,
 1.173978759499455,
 1.127732862468152,
 1.4575150555573075,
 1.9997471535117932,
 1.1381318308249777,
 1.3706227141160197,
 1.3304374299301567,
 1.2423784532034907,
 1.3727362397591147,
 1.6302808113819593,
 1.0687188435270525,
 1.6818473679349888,
 1.506925021095038,
 1.609691325110815,
 1.0021484011306625,
 1.5439762629778633,
 1.745564407314593,
 1.2431062712391936,
 1.4252858947466438,
 1.6551558583379116,
 1.8286177437562117,
 1.8452707105228299,
 1.200137409781724,
 1.4763525039943406,
 1.0813668205205698,
 1.7275268599576041,
 1.1222479789234143,
 1.9139982094538865,
 1.9227377943614794,
 1.1956206291410507,
 1.9401696248895468,
 1.074884832190405,
 1.

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 202 ms, sys: 33.2 ms, total: 235 ms
Wall time: 2.3 s
